In [1]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.columns

Index(['Income', 'Age', 'Dependents', 'Occupation', 'City_Tier', 'Rent',
       'Loan_Repayment', 'Insurance', 'Groceries', 'Transport', 'Eating_Out',
       'Entertainment', 'Utilities', 'Healthcare', 'Education',
       'Miscellaneous', 'Desired_Savings_Percentage', 'Desired_Savings',
       'Disposable_Income', 'Potential_Savings_Groceries',
       'Potential_Savings_Transport', 'Potential_Savings_Eating_Out',
       'Potential_Savings_Entertainment', 'Potential_Savings_Utilities',
       'Potential_Savings_Healthcare', 'Potential_Savings_Education',
       'Potential_Savings_Miscellaneous'],
      dtype='object')

In [4]:
X = df[['Income', 'Age', 'Dependents', 'Rent',
       'Loan_Repayment', 'Insurance', 'Groceries', 'Transport', 'Eating_Out',
       'Entertainment', 'Utilities', 'Healthcare', 'Education',
       'Miscellaneous', 'Desired_Savings_Percentage', 'Desired_Savings',
       'Disposable_Income']]

Y = df[['Potential_Savings_Groceries',
       'Potential_Savings_Transport', 'Potential_Savings_Eating_Out',
       'Potential_Savings_Entertainment', 'Potential_Savings_Utilities',
       'Potential_Savings_Healthcare', 'Potential_Savings_Education',
       'Potential_Savings_Miscellaneous']]

In [5]:
X.head()

,Income,Age,Dependents,Rent,Loan_Repayment,Insurance,Groceries,Transport,Eating_Out,Entertainment,Utilities,Healthcare,Education,Miscellaneous,Desired_Savings_Percentage,Desired_Savings,Disposable_Income
0,44637.249636,49,0,13391.174891,0.000000,2206.490129,6658.768341,2636.970696,1651.801726,1536.184255,2911.792231,1546.914539,0.000000,831.525120,13.890948,6200.537192,11265.627707
1,26858.596592,34,2,5371.719318,0.000000,869.522617,2818.444460,1543.018778,649.378103,1050.241738,1626.143346,1137.351325,1551.723167,564.235007,7.160376,1923.176434,9676.818733
2,50367.605084,35,1,7555.140763,4612.103386,2201.800050,6313.222081,3221.396403,1513.814376,1723.306926,3368.457108,2178.515847,3160.026299,628.371220,13.997808,7050.360422,13891.450624
3,101455.600247,21,0,15218.340037,6809.441427,4889.418087,14690.149363,7106.130005,5040.249158,2858.194240,6128.550387,4571.118217,0.000000,2526.055712,16.455440,16694.965136,31617.953615
4,24875.283548,52,4,4975.056710,3112.609398,635.907170,3034.329665,1276.155163,692.827225,660.186852,1092.689024,1169.095816,1445.215694,515.510300,7.533982,1874.099434,6265.700532


In [6]:
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = pd.DataFrame(scaler_X.fit_transform(X))
Y = pd.DataFrame(scaler_y.fit_transform(Y))

In [7]:
ar1 = ['Income', 'Age', 'Dependents', 'Rent',
       'Loan_Repayment', 'Insurance', 'Groceries', 'Transport', 'Eating_Out',
       'Entertainment', 'Utilities', 'Healthcare', 'Education',
       'Miscellaneous', 'Desired_Savings_Percentage', 'Desired_Savings',
       'Disposable_Income']

ar2 = ['Potential_Savings_Groceries',
       'Potential_Savings_Transport', 'Potential_Savings_Eating_Out',
       'Potential_Savings_Entertainment', 'Potential_Savings_Utilities',
       'Potential_Savings_Healthcare', 'Potential_Savings_Education',
       'Potential_Savings_Miscellaneous']

print(len(ar1))
print(len(ar2))

17
8


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from torch.utils.data import Dataset,DataLoader
from torch.utils.data import TensorDataset


X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)


train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [10]:
class ClassicalANN(nn.Module):

    def __init__(self,in_features=5):
        super().__init__()
        self.fc1 = nn.Linear(17,64)
        self.fc2 = nn.Linear(64,32)
        self.fc7 = nn.Linear(32,8)

    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc7(x)

        return x

In [11]:
import torch.optim as optim

model = ClassicalANN()

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [12]:
# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/50, Loss: 0.3668
Epoch 2/50, Loss: 0.3126
Epoch 3/50, Loss: 0.2997
Epoch 4/50, Loss: 0.2972
Epoch 5/50, Loss: 0.2966
Epoch 6/50, Loss: 0.2931
Epoch 7/50, Loss: 0.2906
Epoch 8/50, Loss: 0.2923
Epoch 9/50, Loss: 0.2877
Epoch 10/50, Loss: 0.2895
Epoch 11/50, Loss: 0.2854
Epoch 12/50, Loss: 0.2867
Epoch 13/50, Loss: 0.2852
Epoch 14/50, Loss: 0.2836
Epoch 15/50, Loss: 0.2821
Epoch 16/50, Loss: 0.2826
Epoch 17/50, Loss: 0.2800
Epoch 18/50, Loss: 0.2791
Epoch 19/50, Loss: 0.2784
Epoch 20/50, Loss: 0.2778
Epoch 21/50, Loss: 0.2783
Epoch 22/50, Loss: 0.2774
Epoch 23/50, Loss: 0.2777
Epoch 24/50, Loss: 0.2754
Epoch 25/50, Loss: 0.2739
Epoch 26/50, Loss: 0.2740
Epoch 27/50, Loss: 0.2738
Epoch 28/50, Loss: 0.2709
Epoch 29/50, Loss: 0.2726
Epoch 30/50, Loss: 0.2701
Epoch 31/50, Loss: 0.2706
Epoch 32/50, Loss: 0.2708
Epoch 33/50, Loss: 0.2695
Epoch 34/50, Loss: 0.2672
Epoch 35/50, Loss: 0.2696
Epoch 36/50, Loss: 0.2657
Epoch 37/50, Loss: 0.2660
Epoch 38/50, Loss: 0.2660
Epoch 39/50, Loss: 0.

In [13]:
# Evaluation
model.eval()
total_loss = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        total_loss += loss.item()
print(f"Test Loss: {total_loss/len(test_loader):.4f}")

Test Loss: 0.3717


In [14]:
torch.save(model.state_dict(),"model.pth")